# Download Localizer Data

Many of the imaging tutorials will use open data from the Pinel Localizer task.

The Pinel Localizer task was designed to probe several different types of basic cognitive processes, such as visual perception, finger tapping, language, and math. Several of the tasks are cued by reading text on the screen (i.e., visual modality) and also by hearing auditory instructions (i.e., auditory modality). The trials are randomized across conditions and have been optimized to maximize efficiency for a rapid event related design. There are 100 trials in total over a 5-minute scanning session. Read the [paper](https://bmcneurosci.biomedcentral.com/articles/10.1186/1471-2202-8-91) for more specific details about the task. You can also view the [Data](http://brainomics.cea.fr/localizer/dataset?rql=Any%20X%2C%20XT%2C%20XL%2C%20XI%2C%20XF%2C%20XD%20WHERE%20X%20is%20Scan%2C%20X%20type%20XT%2C%20X%20label%20XL%2C%20X%20identifier%20XI%2C%20X%20format%20XF%2C%20X%20description%20XD) on the author’s cubicweb interface.

This dataset is well suited for these tutorials as it is (a) publicly available to anyone in the world, (b) relatively small (only about 5min), and (c) provides many options to create different types of contrasts.

There are a total of 94 subjects available, but we will primarily only be working with a smaller subset of about 30.

Downloading the data is very easy.  All we need to do is specify a directory to download and unpack it.

In [54]:
import pandas as pd
from nltools.data import Brain_Data
from nltools.datasets import fetch_localizer

data_dir = '/Users/lukechang/nilearn_data/brainomics_localizer/'

We first will create a list of subject identification numbers from 's01' to 's30'.  Then we will download all of the subjects one by one using the `fetch_localizer` function from `nltools.datasets`.  You can easily modify this code to download more or less subjects.

We can choose to download either the 'raw' or 'preprocessed' data. Feel free to download both, but you will have to change the `data_type` and rerun the function. 

We can also choose if we want to download the anatomical images. This is useful for running preprocessing later. 

If you are eager to get going and don't plan on preprocessing the data yourself, feel free to set `get_anats=False, data_type='preprocessed'` to download the bare minimum. 

Don't forget to set `data_dir=data_dir` if you want to put the data in a specific folder.  Otherwise it will default to `~/nilearn_data/brainomics_localizer/`.


In [33]:
sub_ids = [f'S{x:02}' for x in np.arange(1,31)]

f = fetch_localizer(subject_ids=sub_ids, get_anats=True, data_type='raw')

Downloaded 1555 of 1555 bytes (100.0%,    0.0s remaining) ...done. (1 seconds, 0 min)


Downloaded 64001 of 64001 bytes (100.0%,    0.0s remaining) ...done. (4 seconds, 0 min)


Ok, we have now downloaded all of the data!

The `f` variable contains a dictionary of all of the relevant data.  

 - `f['functional']`: contains a list of the functional data.
 - `f['structural']`: contains a list of the structural data.
 - `f['ext_vars']`: contains all of the subject metadata
 - `f['description']`: contains a brief description of the dataset.

In [52]:
print(f['description'].decode())

Brainomics Localizer


Notes
-----
A protocol that captures the cerebral bases of auditory and
visual perception, motor actions, reading, language comprehension
and mental calculation at an individual level. Individual functional
maps are reliable and quite precise.


Content
-------
    :'func': Nifti images of the neural activity maps
    :'cmaps': Nifti images of contrast maps
    :'tmaps': Nifti images of corresponding t-maps
    :'masks': Structural images of the mask used for each subject.
    :'anats': Structural images of anatomy of each subject

References
----------
For more information about this dataset's structure:
http://brainomics.cea.fr/localizer/

To cite this dataset:
Papadopoulos Orfanos, Dimitri, et al.
"The Brainomics/Localizer database."
NeuroImage 144.B (2017): 309.

For an example of scientific results obtained using this dataset:
Pinel, Philippe, et al.
"Fast reproducible identification and large-scale databasing of
 individual functional cognitive networks."
B

Now let's take a look inside the metadata file.

It is a pandas data frame with info about each participant.



In [ ]:
f['ext_vars'].head()

Ok, now let's try to load one of the functional datasets using `Brain_Data`.

In [56]:
data = Brain_Data(f['functional'][0])

DimensionError: Input data has incompatible dimensionality: Expected dimension is 4D and you provided a 5D image. See http://nilearn.github.io/manipulating_images/input_output.html.

Uh, oh...  Looks like this data was created using an older version of SPM and isn't being read correctly by `Brain_Data`.

Let's do our first bit of debugging and fix this.

We will use nibabel to load the data and inspect the shape of the data file.

In [63]:
import nibabel as nib

dat = nib.load(f['functional'][0][0])
dat.shape

(64, 64, 40, 1, 128)

The first 3 dimensions correspond to X, Y, & Z planes.  The last dimension is time. It looks like there is an extra dimension that we will need to get rid of so that we can load the data correctly.

We will get the data into a numpy array and use the `squeeze()` method to collapse the extra dimension.

In [67]:
dat.get_data().squeeze().shape

(64, 64, 40, 128)

See how we now have the correct shape?

Let's now create a new nibabel instance with the correct dimensions and write over the file.

To create a new nibabel instance, we just need to pass in the 4-D numpy array with the affine matrix.

In [68]:
dat.affine

array([[ -3.,   0.,   0.,  96.],
       [  0.,   3.,   0., -96.],
       [  0.,   0.,   3., -60.],
       [  0.,   0.,   0.,   1.]])

In [75]:
dat_fixed = nib.Nifti1Image(dat.get_data().squeeze(), dat.affine)
nib.save(dat_fixed, f['functional'][0][0])

Let's check and see if we can correctly load the file now.

In [79]:
data = Brain_Data(f['functional'][0])
data

nltools.data.brain_data.Brain_Data(data=(128, 238955), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[])

Great!  Looks like we fixed it.  Let's now fix all of the rest of the images so that we won't have this problem again.

In [89]:
for file_name in f['functional']:
    dat = nib.load(file_name[0])
    if len(dat.shape) > 4:
        dat_fixed = nib.Nifti1Image(dat.get_data().squeeze(), dat.affine)
        nib.save(dat_fixed, file_name[0])